In [1]:
import numpy as np
import pkg_resources
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
from flygym.envs.nmf_mujoco import NeuroMechFlyMuJoCo
from flygym.terrain.mujoco_terrain import GappedTerrain

from tqdm import trange
from flygym.util.config import all_leg_dofs

from scipy.signal import medfilt
from scipy.integrate import ode

import PIL.Image

In [2]:
# Initialize simulation
run_time = 1
out_dir = Path('../CPGs')

#friction between the fly and the ground
friction = 1.0

physics_config = {
    'joint_stiffness': 2500,
    'friction': (friction, 0.005, 0.0001),
    'gravity': (0, 0, -9.81e5)}
terrain_config = {'fly_pos': (200, 200, 600),
                  'friction': (friction, 0.005, 0.0001)}

nmf = NeuroMechFlyMuJoCo(render_mode='saved',
                         timestep=1e-4,
                         render_config={'playspeed': 0.1, 'camera': 'Animat/camera_left_top'},
                         init_pose='stretch',
                         actuated_joints=all_leg_dofs,
                         terrain='gapped', terrain_config=terrain_config)

num_steps_base = int(run_time / nmf.timestep)

In [3]:

legs = ["RF", "RM", "RH", "LF", "LM", "LH"]
n_oscillators = len(legs)

dt = nmf.timestep  # seconds
t = np.arange(0, run_time, dt)

# lets say we want 10 oscillations in the time period
n_steps = 10
frequencies = np.ones(n_oscillators) * n_steps / run_time

# For now each oscillator have the same amplitude
target_amplitude = 1.0
target_amplitudes = np.ones(n_oscillators) * target_amplitude
#rate of change of the amplitude for each oscillator
rate = 10
rates = np.ones(n_oscillators) * rate

# Here we just build a chain of oscillators form 1 to 6
# They all have a bias of pi/6 from one to the other
bias = np.pi/3
# bias only above or bellow the main diagonal => only coupling between neighbouring oscillators
phase_biases = np.diag(np.ones(n_oscillators-1)*bias,k=1) - np.diag(np.ones(n_oscillators-1) * bias, k=-1)

# We wont play with the coupling weights yet so lets set it as the same as the phase biases
# As a consequence the oscillators that have a phase difference of zero are not coupled (depending on your implementation you might want to change that)
coupling_weights = (np.abs(phase_biases) > 0).astype(float) * 10.0 #* 10.0

In [4]:
def phase_oscillator(_time, state):
    """Phase oscillator model used in Ijspeert et al. 2007"""
    
    # current state of the oscillator with phases and amplitudes
    phases = state[:n_oscillators]
    amplitudes = state[n_oscillators:2*n_oscillators]

    # NxN matrix with the phases of the oscillators, each row copy of the phases vector
    phase_matrix = np.tile(phases, (n_oscillators, 1))

    # NxN matrix with the amplitudes of the oscillators, each row copy of the amplitudes vector
    amp_matrix = np.tile(amplitudes, (n_oscillators, 1))

    freq_contribution = 2*np.pi*frequencies

    #  scaling of the phase differences between oscillators by the amplitude of the oscillators and the coupling weights
    scaling = np.multiply(amp_matrix, coupling_weights)

    # phase matrix and transpose substraction are analogous to the phase differences between oscillators, those should be close to the phase biases
    phase_shifts_contribution = np.sin(phase_matrix-phase_matrix.T-phase_biases)

    # Here we compute the contribution of the phase biases to the derivative of the phases
    # we mulitply two NxN matrices and then sum over the columns (all j oscillators contributions) to get a vector of size N 
    coupling_contribution = np.sum(np.multiply(scaling, phase_shifts_contribution), axis=1)

    # Here we compute the derivative of the phases given by the equations defined previously. 
    # We are using for that matrix operations to speed up the computation
    dphases =  freq_contribution + coupling_contribution
    
    damplitudes = np.multiply(rates, target_amplitudes-amplitudes)
    
    return np.concatenate([dphases, damplitudes])

def sine_output(phases, amplitudes):
        return amplitudes * (1 + np.cos(phases))

In [5]:
# Load recorded data
data_path = Path(pkg_resources.resource_filename('flygym', 'data'))
with open(data_path / 'behavior' / 'single_steps.pkl', 'rb') as f:
    data = pickle.load(f)

In [6]:
# Interpolate 5x
step_duration = len(data['joint_LFCoxa'])
# number of samples in the step duration
interp_step_duration = int(step_duration * data['meta']['timestep'] / nmf.timestep)
# holds the interpolated data for all joints
step_data_block_base = np.zeros((len(nmf.actuated_joints), interp_step_duration))
# time at which each joint angle was measured
measure_t = np.arange(step_duration) * data['meta']['timestep']
#time at which we want to interpolate the data
interp_t = np.arange(interp_step_duration) * nmf.timestep

# Interpolate data for each joint at the higher frequency
for i, joint in enumerate(nmf.actuated_joints):
    step_data_block_base[i, :] = np.interp(interp_t, measure_t, data[joint])

In [7]:
step_data_block_manualcorrect = step_data_block_base.copy()

for side in ["L", "R"]:
    step_data_block_manualcorrect[nmf.actuated_joints.index(f"joint_{side}MCoxa")] += np.deg2rad(10) # Protract the midlegs: angle of midleg coxa increased by 10 degrees causing the leg to be more extended -> move forward
    step_data_block_manualcorrect[nmf.actuated_joints.index(f"joint_{side}HFemur")] += np.deg2rad(-5) # Retract the hindlegs: angle of hindleg femur decreased by 5 degrees causing the leg to be more flexed -> move backward
    step_data_block_manualcorrect[nmf.actuated_joints.index(f"joint_{side}HTarsus1")] -= np.deg2rad(15) # Tarsus more parallel to the ground (flexed) (also helps with the hindleg retraction): angle of hindleg tarsus1 decreased by 15 degrees causing the leg to be more flexed 
    step_data_block_manualcorrect[nmf.actuated_joints.index(f"joint_{side}FFemur")] += np.deg2rad(15) # Protract the forelegs (slightly to conterbalance Tarsus flexion): angle of foreleg femur increased by 15 degrees causing the leg to be more extended -> move forward
    step_data_block_manualcorrect[nmf.actuated_joints.index(f"joint_{side}FTarsus1")] -= np.deg2rad(15) # Tarsus more parallel to the ground (flexed) (add some retraction of the forelegs): angle of foreleg tarsus1 decreased by 15 degrees causing the leg to be more flexed

step_data_block_gap = step_data_block_base.copy()

In [63]:
n_joints = len(nmf.actuated_joints)

leg_ids = np.arange(len(legs)).astype(int)
joint_ids = np.arange(n_joints).astype(int)
# Map the id of the joint to the leg it belongs to (usefull to go through the steps for each legs)
match_leg_to_joints = np.array([i  for joint in nmf.actuated_joints for i, leg in enumerate(legs) if leg in joint])


def advancement_transfer(phases, step_dur=interp_step_duration, leg_ids_below_surface=None):
    """From phase define what is the corresponding timepoint in the joint dataset.
    In the case of the oscillator, the period is 2pi, and the step duration is the period of the step.
    We have to match those two."""

    period = 2 * np.pi
    t_indices = np.zeros(len(phases), dtype=int)  # Initialize time indices

    if leg_ids_below_surface is not None:
        t_indices[leg_ids_below_surface] = np.round(np.mod(phases[leg_ids_below_surface] * step_dur / period, step_dur - 1)).astype(int)
        t_indices = t_indices[match_leg_to_joints]

    return t_indices

def advancement_transfer(phases, step_dur=interp_step_duration, leg_ids_below_surface=None):
    """From phase define what is the corresponding timepoint in the joint dataset.
    In the case of the oscillator, the period is 2pi, and the step duration is the period of the step.
    We have to match those two."""

    period = 2 * np.pi
    t_indices = np.zeros(len(phases), dtype=int)  # Initialize time indices

    if leg_ids_below_surface is not None:
        for leg_id in leg_ids_below_surface:
            t_indices[leg_id] = np.round(np.mod(phases[leg_id] * step_dur / period, step_dur - 1)).astype(int)
        t_indices = t_indices[match_leg_to_joints]

    else:
        t_indices = np.round(np.mod(phases * step_dur / period, step_dur - 1)).astype(int)
        t_indices = t_indices[match_leg_to_joints]

    return t_indices



In [9]:
# The bias matrix is define as follow: each line is the i oscillator and each column is the j oscillator couplign goes from i to j
# We express the bias in percentage of cycle 
# rows ad columns correspond to legs of the robot and each value is the phase bias between the two legs. Diagonal is 0 
phase_biases_measured= np.array([[0, 0.425, 0.85, 0.51, 0, 0],
                                  [0.425, 0, 0.425, 0, 0.51, 0],
                                  [0.85, 0.425, 0, 0, 0, 0.51],
                                  [0.51, 0, 0, 0, 0.425, 0.85],
                                  [0, 0.51, 0, 0.425, 0, 0.425],
                                  [0, 0, 0.51, 0.85, 0.425, 0]])
                                
phase_biases_idealized = np.array([[0, 0.5, 1.0, 0.5, 0, 0],
                                   [0.5, 0, 0.5, 0, 0.5, 0],
                                   [1.0, 0.5, 0, 0, 0, 0.5],
                                   [0.5, 0, 0, 0, 0.5, 1.0],
                                   [0, 0.5, 0, 0.5, 0, 0.5],
                                   [0, 0, 0.5, 1.0, 0.5, 0]]) 
# Phase bias of one is the same as zero (1 cycle difference)
# If we would use a phase bias of zero, we would need to change the coupling weight strategy

phase_biases = phase_biases_idealized * 2 * np.pi

coupling_weights = (np.abs(phase_biases) > 0).astype(float) * 10.0 #* 10.0

In [78]:
np.random.seed(42)

#reset the oscillator
_ = nmf.reset()


# Set solver
solver = ode(f=phase_oscillator)
solver.set_integrator('dop853')
initial_values = np.random.rand(2*n_oscillators)
solver.set_initial_value(y=initial_values, t=nmf.curr_time)

n_stabilisation_steps = 1000
num_steps = n_stabilisation_steps + num_steps_base + 2000

phases = np.zeros((num_steps, n_oscillators))
amplitudes = np.zeros((num_steps, n_oscillators))

joint_angles = np.zeros((num_steps, n_joints))

#create an zero array with 6 arrays in it of length num_steps
position_data = np.zeros((6, num_steps))

leg_ids_below_surface = None  # Initialize leg IDs below the surface

for i in range(num_steps):
    res = solver.integrate(nmf.curr_time)
    phase = res[:n_oscillators]
    amp = res[n_oscillators:2*n_oscillators]

    phases[i, :] = phase
    amplitudes[i, :] = amp

    if i > n_stabilisation_steps:
        if leg_ids_below_surface is not None:
            indices = advancement_transfer(phase, leg_ids_below_surface=leg_ids_below_surface)
        else:
            indices = advancement_transfer(phase)

        # Assuming that leg_ids_below_surface is a list
        if leg_ids_below_surface:
            # Only take the first leg for adjustment
            leg_id = leg_ids_below_surface[0]
            leg = legs[leg_id]  # Get the leg name (e.g., "RF", "LF", "RM", etc.)

            # Create a separate step_data_block_manualcorrect for each iteration
            step_data_block_manualcorrect_iter = step_data_block_base.copy()

            # Adjust the angle of the femur and tarsus based on the type of leg
            if leg[1] == "F":  # Front leg
                step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Coxa")] += np.deg2rad(20)
                #step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Femur")] += np.deg2rad(60)
                ##step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Tarsus1")] -= np.deg2rad(5)
            elif leg[1] == "M":  # Middle leg
                step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Coxa")] += np.deg2rad(5)
                step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Femur")] += np.deg2rad(10)
                step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Tarsus1")] -= np.deg2rad(5)
            else:  # Back leg
                step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Coxa")] += np.deg2rad(20)
                #step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Femur")] += np.deg2rad(60)
                #step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Tarsus1")] -= np.deg2rad(5)


            action = {'joints': step_data_block_manualcorrect_iter[joint_ids, 0] + (step_data_block_manualcorrect_iter[joint_ids, indices] - step_data_block_manualcorrect_iter[joint_ids, 0]) * amp[match_leg_to_joints]}
            obs, info = nmf.step(action)
            position_data[:, i] = obs['end_effectors'][2::3]

        else:
            action = {'joints': step_data_block_manualcorrect[joint_ids, 0] + \
                (step_data_block_manualcorrect[joint_ids, indices]-step_data_block_manualcorrect[joint_ids, 0])*amp[match_leg_to_joints]}
            obs, info = nmf.step(action)
            position_data[:, i] = obs['end_effectors'][2::3]

        # If position of the end effector is below the platform, change action to go up
        if any(obs['end_effectors'][2::3] < 1000):
            # Find which position_data subarray is below the platform
            below = np.where(position_data[:, i] < 1000)[0]
            leg_ids_below_surface = below  # Assign the leg IDs below the platform to the leg_ids_below_surface variable
        else:
            leg_ids_below_surface = None  # Reset leg IDs below the surface

    else:
        action = {'joints': step_data_block_manualcorrect[joint_ids, 0]}
        obs, info = nmf.step(action)
        position_data[:, i] = obs['end_effectors'][2::3]

    nmf.render()



nmf.save_video(out_dir / "tripod_gap.mp4")

In [79]:
from ipywidgets import Video
Video.from_file(out_dir / 'tripod_gap.mp4')

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free...')

In [14]:
np.random.seed(42)

# Reset the oscillator
_ = nmf.reset()

# Set solver
solver = ode(f=phase_oscillator)
solver.set_integrator('dop853')
initial_values = np.random.rand(2 * n_oscillators)
solver.set_initial_value(y=initial_values, t=nmf.curr_time)

n_stabilisation_steps = 1000
num_steps = n_stabilisation_steps + num_steps_base #+ 5000

phases = np.zeros((num_steps, n_oscillators))
amplitudes = np.zeros((num_steps, n_oscillators))

joint_angles = np.zeros((num_steps, n_joints))

# Create a zero array with 6 arrays of length num_steps
position_data = np.zeros((6, num_steps))

leg_ids_below_surface = []  # Initialize list of leg IDs below the surface
normal_state = True  # Initialize normal walking state

for i in range(num_steps):
    res = solver.integrate(nmf.curr_time)
    phase = res[:n_oscillators]
    amp = res[n_oscillators:2 * n_oscillators]

    phases[i, :] = phase
    amplitudes[i, :] = amp

    if i > n_stabilisation_steps:
        if leg_ids_below_surface:
            indices = advancement_transfer(phase, leg_ids_below_surface=leg_ids_below_surface)
        else:
            indices = advancement_transfer_all(phase)

        # Create a separate step_data_block_manualcorrect for each iteration
        step_data_block_manualcorrect_iter = step_data_block_base.copy()
        if normal_state:
            action = {'joints': step_data_block_manualcorrect[joint_ids, 0] +
                      (step_data_block_manualcorrect[joint_ids, indices] -
                       step_data_block_manualcorrect[joint_ids, 0]) * amp[match_leg_to_joints]}
            obs, info = nmf.step(action)
            position_data[:, i] = obs['end_effectors'][2::3]
            

            # Check if any end effector position is below the platform
            below_surface = np.where(position_data[:, i] < 1000)[0]
            if below_surface.size > 0:
                leg_ids_below_surface = list(set(leg_ids_below_surface) | set(below_surface))
                normal_state = False  # Enter extended leg state
            else:
                leg_ids_below_surface = []
        else:
            # Adjust the angles of specific joints based on leg type
            for leg_id in leg_ids_below_surface:
                leg = legs[leg_id]  # Get the leg name (e.g., "RF", "LF", "RM", etc.)
                if leg[1] == "M":  # Middle leg
                    step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Coxa")] += np.deg2rad(60)  # Increase the angle of the midleg coxa
                    step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Femur")] += np.deg2rad(90)  # Increase the angle of the midleg femur in order 
                    #step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Tarsus1")] += np.deg2rad(5)  # Decrease the angle of the foreleg tarsus1
                elif leg[1] == "F":  # Front leg
                    step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Femur")] += np.deg2rad(60)  # Increase the angle of the midleg femur
                    #step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Tarsus1")] -= np.deg2rad(5)  # Decrease the angle of the foreleg tarsus1
                else:
                    
                    step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Femur")] += np.deg2rad(60)
                    #step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Tarsus1")] -= np.deg2rad(5)

            action = {'joints': step_data_block_manualcorrect_iter[joint_ids, 0] +
                      (step_data_block_manualcorrect_iter[joint_ids, indices] -
                       step_data_block_manualcorrect_iter[joint_ids, 0]) * amp[match_leg_to_joints]}
            obs, info = nmf.step(action)
            position_data[:, i] = obs['end_effectors'][2::3]
            
            # Check if the legs have extended sufficiently
            if all(obs['end_effectors'][2::3] >= 1000):
                leg_ids_below_surface = []
                normal_state = True  # Enter normal walking state

    else:
        action = {'joints': step_data_block_manualcorrect[joint_ids, 0]}
        obs, info = nmf.step(action)
        position_data[:, i] = obs['end_effectors'][2::3]
        

    nmf.render() 

nmf.save_video(out_dir / "tripod_gap_2.mp4")

In [55]:
from ipywidgets import Video
Video.from_file(out_dir / 'tripod_gap_2.mp4')

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free...')

In [78]:
np.random.seed(42)

# Reset the oscillator
_ = nmf.reset()

# Set solver
solver = ode(f=phase_oscillator)
solver.set_integrator('dop853')
initial_values = np.random.rand(2 * n_oscillators)
solver.set_initial_value(y=initial_values, t=nmf.curr_time)

n_stabilisation_steps = 1000
num_steps = n_stabilisation_steps + num_steps_base

phases = np.zeros((num_steps, n_oscillators))
amplitudes = np.zeros((num_steps, n_oscillators))

joint_angles = np.zeros((num_steps, n_joints))

# Create a zero array with 6 arrays of length num_steps
position_data = np.zeros((6, num_steps))

leg_ids_below_surface = []  # Initialize list of leg IDs below the surface
normal_state = True  # Initialize normal walking state

for i in range(num_steps):
    res = solver.integrate(nmf.curr_time)
    phase = res[:n_oscillators]
    amp = res[n_oscillators:2 * n_oscillators]

    phases[i, :] = phase
    amplitudes[i, :] = amp

    if i > n_stabilisation_steps:
        if leg_ids_below_surface:
            indices = advancement_transfer(phase, leg_ids_below_surface=leg_ids_below_surface)
        else:
            indices = advancement_transfer_all(phase)

        # Create a separate step_data_block_manualcorrect for each iteration
        step_data_block_manualcorrect_iter = step_data_block_manualcorrect.copy()

        if normal_state:
            action = {'joints': step_data_block_manualcorrect[joint_ids, 0] +
                      (step_data_block_manualcorrect[joint_ids, indices] -
                       step_data_block_manualcorrect[joint_ids, 0]) * amp[match_leg_to_joints]}
            obs, info = nmf.step(action)
            position_data[:, i] = obs['end_effectors'][2::3]

            # Check if any end effector position is below the platform
            below_surface = np.where(position_data[:, i] < 1000)[0]
            if below_surface.size > 0:
                leg_ids_below_surface = list(set(leg_ids_below_surface) | set(below_surface))
                normal_state = False  # Enter extended leg state
            else:
                leg_ids_below_surface = []
        else:
            # Adjust the angles of specific joints based on leg type
            for leg_id in leg_ids_below_surface:
                leg = legs[leg_id]  # Get the leg name (e.g., "RF", "LF", "RM", etc.)
                if leg[1] == "M":  # Middle leg
                    step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Coxa")] += np.deg2rad(60)  # Increase the angle of the midleg coxa
                    step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Femur")] += np.deg2rad(90)  # Increase the angle of the midleg femur in order 
                    #step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Tarsus1")] += np.deg2rad(5)  # Decrease the angle of the foreleg tarsus1
                elif leg[1] == "F":  # Front leg
                    step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Femur")] += np.deg2rad(60)  # Increase the angle of the midleg femur
                    #step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Tarsus1")] -= np.deg2rad(5)  # Decrease the angle of the foreleg tarsus1
                else:
                    step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Femur")] += np.deg2rad(60)
                    #step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Tarsus1")] -= np.deg2rad(5)

            action = {'joints': step_data_block_manualcorrect_iter[joint_ids, 0] +
                      (step_data_block_manualcorrect_iter[joint_ids, indices] -
                       step_data_block_manualcorrect_iter[joint_ids, 0]) * amp[match_leg_to_joints]}
            obs, info = nmf.step(action)
            position_data[:, i] = obs['end_effectors'][2::3]

            # Check if the legs have extended sufficiently
            if all(obs['end_effectors'][2::3] >= 1000):
                leg_ids_below_surface = []
                normal_state = True  # Enter normal walking state

    else:
        action = {'joints': step_data_block_manualcorrect[joint_ids, 0]}
        obs, info = nmf.step(action)
        position_data[:, i] = obs['end_effectors'][2::3]
        
    nmf.render()

nmf.save_video(out_dir / "tripod_gap_3.mp4")

AttributeError: 'list' object has no attribute 'size'

In [57]:
from ipywidgets import Video
Video.from_file(out_dir / 'tripod_gap_3.mp4')

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free...')

In [50]:
def advancement_transfer(phases, leg_ids_below_surface=None, step_dur=interp_step_duration):
    """From phase define what is the corresponding timepoint in the joint dataset.
    In the case of the oscillator, the period is 2pi, and the step duration is the period of the step.
    We have to match those two."""

    period = 2 * np.pi
    t_indices = np.zeros(len(phases), dtype=int)  # Initialize time indices

    # only adjust the indices of legs not on the block
    if len(leg_ids_below_surface) > 0:  
        t_indices[leg_ids_below_surface] = np.round(np.mod(phases[leg_ids_below_surface] * step_dur / period, step_dur - 1)).astype(int)
    
    # adjust the indices according to the corresponding joints
    t_indices = t_indices[match_leg_to_joints]

    return t_indices



In [55]:
block_height_threshold = 1010

np.random.seed(42)

_ = nmf.reset()

solver = ode(f=phase_oscillator)
solver.set_integrator('dop853')
initial_values = np.random.rand(2*n_oscillators)
solver.set_initial_value(y=initial_values, t=nmf.curr_time)

n_stabilisation_steps = 1000
num_steps = n_stabilisation_steps + num_steps_base

phases = np.zeros((num_steps, n_oscillators))
amplitudes = np.zeros((num_steps, n_oscillators))

joint_angles = np.zeros((num_steps, n_joints))

obs_list_tripod = []

# add an array to store the position data of the feet
position_data = np.zeros((6, num_steps))


for i in range(num_steps):
    res = solver.integrate(nmf.curr_time)
    phase = res[:n_oscillators]
    amp = res[n_oscillators:2*n_oscillators]

    phases[i, :] = phase
    amplitudes[i, :] = amp 

    if i > n_stabilisation_steps:
        # Get legs not on block
        legs_below_surface = np.where(position_data[:, i] < block_height_threshold)[0]
        
        indices = advancement_transfer(phase, legs_below_surface)

        # Create a separate step_data_block_manualcorrect for each iteration
        step_data_block_manualcorrect_iter = step_data_block_base.copy()
        
        # Update step_data_block_manualcorrect_iter for each leg not on block
        for leg_id in legs_below_surface:
            leg = legs[leg_id]
            
            if leg[1] == "M":  # Middle leg
                #step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Coxa")] += np.deg2rad(-10)  # Increase the angle of the midleg coxa
                step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Femur")] += np.deg2rad(-10)  # Increase the angle of the midleg femur in order 
                    #step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Tarsus1")] += np.deg2rad(5)  # Decrease the angle of the foreleg tarsus1
            elif leg[1] == "F":  # Front leg
                step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Femur")] += np.deg2rad(60)  # Increase the angle of the midleg femur
                    #step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Tarsus1")] -= np.deg2rad(5)  # Decrease the angle of the foreleg tarsus1
            else:
                step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Femur")] += np.deg2rad(60)
                    #step_data_block_manualcorrect_iter[nmf.actuated_joints.index(f"joint_{leg}Tarsus1")] -= np.deg2rad(5)

        action = {'joints': step_data_block_manualcorrect_iter[joint_ids, 0] +
                (step_data_block_manualcorrect_iter[joint_ids, indices] -
                step_data_block_manualcorrect_iter[joint_ids, 0]) * 2 * amp[match_leg_to_joints]}
    else:
        action = {'joints': step_data_block_manualcorrect[joint_ids, 0]}
        
    obs, info = nmf.step(action)

    # Store the position data of the feet and thorax (z-axis only)
    position_data[:, i] = obs['end_effectors'][2::3]

    obs_list_tripod.append(obs)
    joint_angles[i, :] = action['joints']
    
    nmf.render()

nmf.save_video(out_dir / "tripod_block_3.mp4")

In [56]:
from ipywidgets import Video
Video.from_file(out_dir / 'tripod_gap_3.mp4')

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free...')